In [1]:
import pandas as pd
import re

In [6]:
store = 'zehrs'

dairy = pd.read_csv(f'raw_data/{store}/dairy_and_eggs.csv')
bakery = pd.read_csv(f'raw_data/{store}/bakery.csv')
meat = pd.read_csv(f'raw_data/{store}/meat.csv')
produce = pd.read_csv(f'raw_data/{store}/produce.csv')
seafood = pd.read_csv(f'raw_data/{store}/seafood.csv')

raw_data = pd.DataFrame()
raw_data = raw_data.append(dairy)
raw_data = raw_data.append(bakery)
raw_data = raw_data.append(meat)
raw_data = raw_data.append(produce)
raw_data = raw_data.append(seafood)
raw_data['store'] = store

In [7]:
raw_data.head()

,category,brand,product_text,product_name,price_text,sale_price,per_unit_price_text,is_sale,store
0,dairy_and_eggs,No Name,Large Grade A Eggs,No NameLarge Grade A Eggs12 eggs,$3.89ea,NaN,$0.32/ 1ea,False,zehrs
1,dairy_and_eggs,Neilson,Partly Skimmed Milk 2% MF,NeilsonPartly Skimmed Milk 2% MF4 l,$5.69ea,NaN,$0.14/ 100ml,False,zehrs
2,dairy_and_eggs,Neilson,Half and Half Cream,NeilsonHalf and Half Cream1 l,$3.99ea,NaN,$0.40/ 100ml,False,zehrs
3,dairy_and_eggs,No Name,"White Large Eggs, 30 Pack","No NameWhite Large Eggs, 30 Pack 30 ea",$9.49ea,NaN,$0.32/ 1ea,False,zehrs
4,dairy_and_eggs,Neilson,Partly Skimmed Milk 1% MF,NeilsonPartly Skimmed Milk 1% MF4 l,$5.69ea,NaN,$0.14/ 100ml,False,zehrs


In [12]:
clean_data = pd.DataFrame(columns = ['store', 'category', 'brand', 'product', 'price', 'sale_price'
                , 'per_unit_price', 'sale_per_unit_price', 'units', 'is_sale'])

for index, row in raw_data.iterrows():

    is_sale = row.is_sale 
    
    if is_sale:
        price_was = row.price_text
        price = row.sale_price
    else:
        price = row.price_text
        price_was = price

    price = float(re.findall('(([0-9]+)\.([0-9]+))', price)[0][0])
    price_was = float(re.findall('(([0-9]+)\.([0-9]+))', price_was)[0][0])
    
    per_unit_price_text = row.per_unit_price_text

    if per_unit_price_text == per_unit_price_text: 
        # pup, unit = find_per_unit_price(per_unit_price_text)
        pup = float(per_unit_price_text.split('/ ')[0].replace('$', ''))
        unit = per_unit_price_text.split('/ ')[1]

        # pup for was_price 
        scaling_factor = price / pup
        pup_was = price_was / scaling_factor
    else: # no units
        pup, pup_was, unit = None, None, None


    if not is_sale:
        price, pup = None, None
    
    clean_data = clean_data.append({
                    'store': store, 
                    'category': row.category,
                    'brand': row.brand, 
                    'product': row.product_text, 
                    'price': price_was,
                    'sale_price':  price,
                    'per_unit_price': pup_was,
                    'sale_per_unit_price': pup,
                    'units': unit,
                    'is_sale': is_sale
                }, ignore_index = True)


In [13]:
clean_data

,store,category,brand,product,price,sale_price,per_unit_price,sale_per_unit_price,units,is_sale
0,zehrs,dairy_and_eggs,No Name,Large Grade A Eggs,3.89,None,0.32,None,1ea,False
1,zehrs,dairy_and_eggs,Neilson,Partly Skimmed Milk 2% MF,5.69,None,0.14,None,100ml,False
2,zehrs,dairy_and_eggs,Neilson,Half and Half Cream,3.99,None,0.4,None,100ml,False
3,zehrs,dairy_and_eggs,No Name,"White Large Eggs, 30 Pack",9.49,None,0.32,None,1ea,False
4,zehrs,dairy_and_eggs,Neilson,Partly Skimmed Milk 1% MF,5.69,None,0.14,None,100ml,False
...,...,...,...,...,...,...,...,...,...,...
2973,zehrs,seafood,President's Choice,Wild Atlantic Cold Water Shrimp Cooked Peeled,13.99,None,4.11,None,100g,False
2974,zehrs,seafood,Seaquest,Crab Leg Style,5.99,None,1.32,None,100g,False
2975,zehrs,seafood,President's Choice,Smoked Atlantic Salmon,11.99,None,7.99,None,100g,False
2976,zehrs,seafood,Anchor Bay,Cooked Clam Meat,6.99,None,2.33,None,100g,False
